In [146]:
raw_data = pd.read_csv('./data/raw_data.csv', index_col=0)

In [147]:
raw_data.head()

,연,월,일,시간대,요일,기온,강수량,풍속,습도,적설량,미세먼지,초미세먼지,공휴일,통화건수
0,2016,19.4,1,0,73,NaN,0.0,NaN,NaN,0.0,48.0,26.0,0,67.0
1,2016,10.0,1,1,토,18.8,0.0,1.3,77.0,0.0,48.0,26.0,0,27.0
2,2016,10.0,1,2,토,18.5,0.0,0.4,78.0,0.0,48.0,26.0,0,16.0
3,2016,10.0,1,3,토,18.3,0.0,0.4,79.0,0.0,48.0,26.0,0,10.0
4,2016,10.0,1,4,토,18.1,0.0,1.3,81.0,0.0,48.0,26.0,0,18.0


In [148]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(raw_data, test_size=0.3,random_state=11)

In [149]:
delivery = train_set.drop(["통화건수"], axis=1)
delivery_labels = train_set['통화건수'].copy()

In [150]:
# 카테고리데이터 인코딩

In [151]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

data_cat = ['연', '월', '일', '시간대', '요일', '공휴일']

cat_pipeline = ColumnTransformer([
    ('cat', OneHotEncoder(), data_cat),
])

delivery_prepared = cat_pipeline.fit_transform(delivery)

In [152]:
delivery_prepared.shape

(18396, 88)

In [153]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(delivery_prepared, delivery_labels)

LinearRegression()

In [154]:
# 5개 데이터 예측
some_data = delivery.iloc[:5]
some_label = delivery_labels.iloc[:5]
some_data_prepared = cat_pipeline.transform(some_data)

a = [ 0 for x in lin_reg.predict(some_data_prepared) if x <= 0]


print("예측:", lin_reg.predict(some_data_prepared))

예측: [20.97646327 54.0358424  11.45475707 38.31262555  0.9857099 ]


In [155]:
# 실제값과 비교
print("레이블:", list(some_label))

레이블: [28.0, 64.0, 0.0, 37.0, 15.0]


In [156]:
from sklearn.metrics import mean_squared_error

delivery_predictions = lin_reg.predict(delivery_prepared)
lin_mse = mean_squared_error(delivery_labels, delivery_predictions)
lim_rmse = np.sqrt(lin_mse)
lim_rmse

18.514557633662463

In [157]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(delivery_labels, delivery_predictions)
lin_mae

11.91888709724295

In [158]:
from sklearn.metrics import r2_score
r2_score(delivery_labels, delivery_predictions)

0.8696628830247541

In [159]:
# Decsion Tree
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=11)
tree_reg.fit(delivery_prepared, delivery_labels)

DecisionTreeRegressor(random_state=11)

In [160]:
delivery_predictions = tree_reg.predict(delivery_prepared)
tree_mse = mean_squared_error(delivery_labels, delivery_predictions)
tree_rmse = np.sqrt(tree_mse)
print("tree_rmse: ", tree_rmse)
print("r2_score: ", r2_score(delivery_labels, delivery_predictions))

tree_rmse:  0.20332365530420268
r2_score:  0.9999842812774669


In [161]:
# k-fold 교차검증준비
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, delivery_prepared, delivery_labels, scoring = "neg_mean_squared_error", cv=5)
tree_rmse_scores = np.sqrt(-scores)

In [162]:
def display_socres(scores):
    print("점수:", scores)
    print("평균:", scores.mean())
    print("표준편차:", scores.std())
    
display_socres(tree_rmse_scores)

점수: [17.65121778 17.16715861 17.40975254 18.42153862 18.05441625]
평균: 17.740816759577534
표준편차: 0.44909733054233747


In [163]:
# 선형회귀 평가 
lin_scores = cross_val_score(lin_reg, delivery_prepared, delivery_labels, scoring = "neg_mean_squared_error", cv=5)
lin_rmse_scores = np.sqrt(-lin_scores)
display_socres(lin_rmse_scores)

점수: [18.15269033 17.36852721 18.94222317 19.53186487 19.00092463]
평균: 18.599246042372776
표준편차: 0.7566819307297752


In [164]:
# random forest
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=50,random_state=11)
forest_reg.fit(delivery_prepared, delivery_labels)
delivery_predictions = forest_reg.predict(delivery_prepared)
forest_mse = mean_squared_error(delivery_labels, delivery_predictions)
forest_rmse = np.sqrt(forest_mse)
print("tree_rmse: ", forest_rmse)
print("r2_score: ", r2_score(delivery_labels, delivery_predictions))

tree_rmse:  5.575197010629102
r2_score:  0.9881815133855016


In [165]:
for_scores = cross_val_score(forest_reg, delivery_prepared, delivery_labels, scoring = "neg_mean_squared_error", cv=5)
forest_rmse_scores = np.sqrt(-for_scores)

In [166]:
display_socres(forest_rmse_scores)

점수: [14.38985472 13.90934356 16.03626959 16.04364482 15.80939994]
평균: 15.23770252531321
표준편차: 0.9052594886010608
